<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/Rescrape_using_siddgood_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping itunes podcast descriptions and reviews

For AIPI540 recommender system module project, getting information on podcasts using similar code to what was presented in https://github.com/siddgood/podcast-recommendation-engine but also scraping reviews and authors of reviews.

In [1]:
!pip install unidecode selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.6 MB/s eta 0:00:00


In [2]:
from bs4 import BeautifulSoup
import requests
import json

import pandas as pd
import numpy as np

import json
import re
import unidecode
import time
from tqdm import tqdm

from selenium import webdriver



Getting list of podcasts from the various categories on the main apple page. 


In [3]:
CATEGORIES = {
    'arts': "https://podcasts.apple.com/us/genre/podcasts-arts/id1301",
    'business': "https://podcasts.apple.com/us/genre/podcasts-business/id1321",
    'comedy': "https://podcasts.apple.com/us/genre/podcasts-comedy/id1303",
    'education': "https://podcasts.apple.com/us/genre/podcasts-education/id1304",
    'fiction': "https://podcasts.apple.com/us/genre/podcasts-fiction/id1483",
    'government': "https://podcasts.apple.com/us/genre/podcasts-government/id1511",
    'health': "https://podcasts.apple.com/us/genre/podcasts-health-fitness/id1512",
    'history': "https://podcasts.apple.com/us/genre/podcasts-history/id1487",
    'kids_and_family': "https://podcasts.apple.com/us/genre/podcasts-kids-family/id1305",
    'leisure': "https://podcasts.apple.com/us/genre/podcasts-leisure/id1502",
    'music': "https://podcasts.apple.com/us/genre/podcasts-music/id1310",
    'news': "https://podcasts.apple.com/us/genre/podcasts-news/id1489",
    'religion_and_spirituality': "https://podcasts.apple.com/us/genre/podcasts-religion-spirituality/id1314",
    'science': "https://podcasts.apple.com/us/genre/podcasts-science/id1533",
    'society_and_culture': "https://podcasts.apple.com/us/genre/podcasts-society-culture/id1324",
    'sports': "https://podcasts.apple.com/us/genre/podcasts-sports/id1545",
    'tv_and_film': "https://podcasts.apple.com/us/genre/podcasts-tv-film/id1309",
    'technology': "https://podcasts.apple.com/us/genre/podcasts-technology/id1318",
    'true_crime': "https://podcasts.apple.com/us/genre/podcasts-true-crime/id1488"
}

In [5]:
all_podcast_links = list()

for category_url in CATEGORIES.values():
    response = requests.get(category_url, timeout=5)
    content = BeautifulSoup(response.content, "lxml")
    podcast_links = content.find('div', class_='grid3-column')

    for link in podcast_links.findAll('a'):
        all_podcast_links.append(link.get('href'))

all_podcast_links = list(set(all_podcast_links))

print('Number of podcast links', len(all_podcast_links))

with open('podcast_links.json', 'w') as outfile:
    json.dump(all_podcast_links, outfile)

Number of podcast links 4560


Utilities for converting 10K, 5M to actual integers for the total number of star reviews, as well as cleaning the text for the titles and the episode descriptions

In [24]:
def convert_si_to_number(x):
    total_stars = 0
    if 'K' in x:
        if len(x) > 1:
            total_stars = float(x.replace('K', '')) * 1000  # convert K to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000  # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000  # convert B to a Billion
    else:
        total_stars = int(x)  # Less than 1000
    return int(total_stars)


def clean_title(t):
    t = unidecode.unidecode(t)
    t = t.replace('\n', ' ')
    t = re.sub(r'[^\w\s]', '', t)
    t = re.sub(r'\d+', '', t)
    t = t.lower()
    t = t.strip()
    return t

def clean_description(dlist):
    
    new_list = []
    for d in dlist:
      d = unidecode.unidecode(d)
      d = d.replace('\n', ' ')
      d = re.sub(r'[^\w\s]', '', d)
      d = re.sub(r'\d+', '', d)
      if re.findall(r'(.*) brought to you by.*', d):
        d = re.sub(r'brought to you by.*', '', d)
      if re.search(r'(.*) sponsored by.*', d):
        d = re.sub(r'sponsored by.*', '', d)
      d = d.lower()
      d = d.strip()
      new_list.append(d)
    return new_list


Build a json object from the scraping of each link and dump into a json file for later processing

In [15]:

def build_podcast_object(link):
    try:
        response = requests.get(link, timeout=10)
        content = BeautifulSoup(response.content, "lxml")
        try:
            title = content.find('h1').find('span').text.strip()
        except AttributeError:
            title = 'NA'
        try:
            producer = content.find('h1').find('a').text.strip()
        except AttributeError:
            try:
                producer = content.find('h1').find('span', class_='product-header__identity podcast-header__identity') \
                    .text.strip()
            except AttributeError:
                producer = 'NA'
        try:
            genre = content.find('li', class_="product-header__list__item").text.strip()
        except AttributeError:
            genre = 'NA'
        try:
            description = content.find('div', class_="product-hero-desc product-hero-desc--side-bar").text.strip()
        except AttributeError:
            description = 'NA'
        try:
            num_episodes = int(content.find('div', class_="product-artwork__caption small-hide medium-show")
                               .text.strip().strip('episodes'))
        except AttributeError:
            num_episodes = 'NA'
        try:
            rating = float(content.find('span', class_="we-customer-ratings__averages__display").text.strip())
        except AttributeError:
            rating = 'NA'
        try:
            num_reviews = convert_si_to_number(content
                                               .find('div', class_="we-customer-ratings__count small-hide medium-show")
                                               .text.strip().strip('Ratings'))
        except AttributeError:
            num_reviews = 'NA'

        try:
            script_tag = content.find('script', {'name': 'schema:podcast-show', 'type': 'application/ld+json'})

            # Extract and load the JSON data
            json_data = json.loads(script_tag.string)

            # Access the features in the JSON data
            script = {'jname': json_data['name'], 
                      'jauthor': json_data['author'],
                      'jdescription': json_data['description'],        
                      'jdate_published': json_data['datePublished'],
                      'joffers': json_data['offers'],
                      'jreviews': json_data['review'],
                      'work_examples' : json_data['workExample']}


        except AttributeError:
            script = 'NA'
            jreviews = 'NA'
            work_examples = 'NA'

    except Exception:
        title = 'NA'
        producer = 'NA'
        genre = 'NA'
        description = 'NA'
        num_episodes = 'NA'
        rating = 'NA'
        num_reviews = 'NA'
        script = 'NA'
    podcast_object = {
        'title': title,
        'producer': producer,
        'genre': genre,
        'description': script['jdescription'],
        'num_episodes': num_episodes,
        'rating': rating,
        'num_reviews': num_reviews,
        'link': link,
        'itunes_id': link.split('/')[-1].strip('id'),
        'reviews': script['jreviews'],
        'epidsode_descriptions': [x['description'] for x in script['work_examples']],
        'ratings_lists': [(x['author'], x['reviewRating']['ratingValue']) for x in script['jreviews']]
    }
    return podcast_object


Making sure the updated scraping to get the episode description text and the reviews/reviewer pairs actually works on a single link before looping through all the links:

In [16]:
podcast_link = all_podcast_links[100]
podcast_link

'https://podcasts.apple.com/us/podcast/renewing-your-mind-with-r-c-sproul/id110916650'

In [17]:
sample_podcast = build_podcast_object(podcast_link)
sample_podcast

{'title': 'Renewing Your Mind with R.C. Sproul',
 'producer': 'Ligonier Ministries',
 'genre': 'Religion & Spirituality',
 'description': 'Since 1994, this daily broadcast has provided accessible, in-depth Bible teaching to millions around the world. As the principal outreach of Ligonier Ministries, it has instructed listeners in the life-changing truths of historic Christianity and has kept the church community informed about pertinen…',
 'num_episodes': 120,
 'rating': 4.8,
 'num_reviews': 3900,
 'link': 'https://podcasts.apple.com/us/podcast/renewing-your-mind-with-r-c-sproul/id110916650',
 'itunes_id': '110916650',
 'reviews': [{'@type': 'Review',
   'author': 'Nerdhoff',
   'datePublished': 'Mar 29, 2023',
   'name': 'Reformation series',
   'reviewRating': {'@type': 'Rating', 'ratingValue': 5}},
  {'@type': 'Review',
   'author': 'Jesusfollower9',
   'datePublished': 'Mar 24, 2023',
   'name': 'Great theology',
   'reviewRating': {'@type': 'Rating', 'ratingValue': 5}},
  {'@type'

In [13]:
# for review in sample_podcast['reviews']:
#     author = review['author']
#     rating = review['reviewRating']['ratingValue']
#     print(f"Author: {author}, Rating: {rating}")

Author: Nerdhoff, Rating: 5
Author: Jesusfollower9, Rating: 5
Author: 999999gd, Rating: 5
Author: Corey Lambrecht, Rating: 4
Author: Still So Much to Learn, Rating: 5
Author: Ccatd, Rating: 5
Author: AfRaven1237, Rating: 5
Author: jlm521, Rating: 5
Author: jakespex, Rating: 5
Author: Paulseywall-z, Rating: 5


In [18]:
counter = 1
podcastObjects = list()
with open('podcast_links.json') as json_file:
    podcasts = json.load(json_file)
    for podcast_link in podcasts:
        try:
            podcast = build_podcast_object(podcast_link)
            podcastObjects.append(podcast)
            print('Completed Podcast: ' + str(counter))
            counter += 1
        except Exception:
            print('Failed on ' + podcast_link)
            counter += 1
            pass

with open('my_scrape_podcast_with_reviews.json', 'w') as outfile:
    json.dump(podcastObjects, outfile)

Completed Podcast: 1
Completed Podcast: 2
Completed Podcast: 3
Completed Podcast: 4
Completed Podcast: 5
Completed Podcast: 6
Completed Podcast: 7
Completed Podcast: 8
Failed on https://podcasts.apple.com/us/podcast/the-ringer-nfl-show/id1109282822
Completed Podcast: 10
Completed Podcast: 11
Completed Podcast: 12
Completed Podcast: 13
Failed on https://podcasts.apple.com/us/podcast/what-next-daily-news-and-analysis/id1438906889
Completed Podcast: 15
Completed Podcast: 16
Completed Podcast: 17
Completed Podcast: 18
Failed on https://podcasts.apple.com/us/podcast/ksr/id1251078708
Completed Podcast: 20
Completed Podcast: 21
Completed Podcast: 22
Completed Podcast: 23
Completed Podcast: 24
Completed Podcast: 25
Completed Podcast: 26
Completed Podcast: 27
Completed Podcast: 28
Failed on https://podcasts.apple.com/us/podcast/duncd-on-basketball-nba-podcast/id986901174
Completed Podcast: 30
Completed Podcast: 31
Completed Podcast: 32
Completed Podcast: 33
Completed Podcast: 34
Failed on https

In [25]:
# !pip install selenium
# !apt-get update 
# !apt install chromium-chromedriver
# !pip install kora -q

In [ ]:
# def get_recent_podcast_episodes(link):
#     episode_titles = ''
#     episode_desc = ''

#     from kora.selenium import wd
#     wd.get(link)
    
#     html = wd.page_source.encode('utf-8')

#     soup = BeautifulSoup(html, 'lxml')
#     text = str(soup.find('script'))

#     try:
#         text = text.split('"workExample":')[1].split(',"aggregateRating"')[0]
#         episode_data = json.loads(text)

#         for episode in episode_data:
#             title = episode['name']
#             c_title = clean_title(title)

#             description = episode['description']
#             c_description = clean_description(description)

#             episode_titles += (c_title + " ")
#             episode_desc += (c_description + " ")

#         episode_titles = episode_titles.strip()
#         episode_desc = episode_desc.strip()

#     except Exception:
#         episode_title = np.nan
#         episode_desc = np.nan
#         print("Failed on: " + str(link))

#     return [episode_titles, episode_desc]

In [26]:
with open('my_scrape_podcast_with_reviews.json') as json_file:
        podcasts = json.load(json_file)

podcasts_df = pd.DataFrame(podcasts)

podcasts_df = podcasts_df[['title', 'producer', 'genre', 'description', 'num_episodes',
                               'rating', 'num_reviews', 'link', 'itunes_id', 'epidsode_descriptions', 'ratings_lists']]
podcasts_df = podcasts_df.replace('NA', np.nan)
#podcasts_df = podcasts_df.dropna()

print(podcasts_df.head())

podcast_titles = list(podcasts_df['title'])
podcast_titles = [title.replace(" ", "") for title in podcast_titles]
podcast_titles = [re.sub(r'[^\w\s]', '', title) for title in podcast_titles]
is_english = [bool(re.match("^[A-Za-z9]*$", title)) for title in podcast_titles]
podcasts_df['is_english'] = is_english
podcasts_df = podcasts_df[podcasts_df.is_english == True]
podcasts_df = podcasts_df.drop(columns=['is_english'])
#podcasts_df['episode_titles'], podcasts_df['episode_descriptions'] = podcasts_df.link.apply(get_recent_podcast_episodes)


podcasts_df = podcasts_df.reset_index(drop=True)





                                               title              producer  \
0  One Strange Thing: Paranormal & True-Weird Mys...     One Strange Thing   
1                                       BibleProject  BibleProject Podcast   
2                        The Domonique Foxworth Show                  ESPN   
3                                     Hacking Humans        CyberWire Inc.   
4                                          Leader Up                  AMSC   

                     genre                                        description  \
0                  History  Paranormal, unexplainable, and uncanny stories...   
1  Religion & Spirituality  The creators of BibleProject have in-depth con...   
2                   Sports  With episodes every Tuesday and Thursday durin...   
3               Technology  Deception, influence, and social engineering i...   
4               Government  Leader Up, a podcast by the Army Management St...   

   num_episodes  rating  num_reviews  \
0   

In [27]:
podcasts_df

,title,producer,genre,description,num_episodes,rating,num_reviews,link,itunes_id,epidsode_descriptions,ratings_lists
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[[RobinFerris, 5], [Pops.99, 1], [ReddEye81, 5..."
1,BibleProject,BibleProject Podcast,Religion & Spirituality,The creators of BibleProject have in-depth con...,352,4.9,15000.0,https://podcasts.apple.com/us/podcast/biblepro...,1050832450,"[David was Israel’s greatest king, but even he...","[[nina52475, 5], [simon tadesse, 5], [HMHuntle..."
2,The Domonique Foxworth Show,ESPN,Sports,With episodes every Tuesday and Thursday durin...,70,4.9,1100.0,https://podcasts.apple.com/us/podcast/the-domo...,1642566714,"[Domonique, Charlie, and Ashley Foxworth along...","[[nick ndd as mm name, 5], [StuArmy910, 5], [L..."
3,Hacking Humans,CyberWire Inc.,Technology,"Deception, influence, and social engineering i...",415,4.7,255.0,https://podcasts.apple.com/us/podcast/hacking-...,1391915810,"[Kathleen Smith, CMO from ClearedJobs.Net sits...","[[FreshDoughnuts, 5], [Jedi Wannabi, 5], [Conj..."
4,Leader Up,AMSC,Government,"Leader Up, a podcast by the Army Management St...",52,5.0,14.0,https://podcasts.apple.com/us/podcast/leader-u...,1378682853,[MSC's Mr. David Howey meets with CSM Jason C....,"[[Nolikeynewudatey, 5]]"
...,...,...,...,...,...,...,...,...,...,...,...
4032,Tales from the Stinky Dragon,Rooster Teeth,Leisure,A D&amp;D podcast from Rooster Teeth! Our brav...,101,4.9,781.0,https://podcasts.apple.com/us/podcast/tales-fr...,1563814788,"[With Asafee on his deathbed, the four Chosen ...","[[moochwaller, 5], [Sakurablossom87, 5], [Cade..."
4033,Morning Microdose,Almost 30,Education,The fact that you came across Morning Microdos...,159,5.0,187.0,https://podcasts.apple.com/us/podcast/morning-...,1639123211,[Drop in for this mind-expanding conversation ...,"[[Blue Sparrow Events, 5], [katikroy3, 5], [Ha..."
4034,Presidential,Washington Post Audio,History,The Washington Post's Presidential podcast exp...,52,4.4,3500.0,https://podcasts.apple.com/us/podcast/presiden...,1072170823,"[Students, teachers and historians reflect on ...","[[ShipShore, 5], [Cool Firework🧨, 5], [John Ve..."
4035,Badlands Cola | A Strange Audio Drama,Renee Taylor Klint,Fiction,Badlands Cola is a cinematic mystery/horror au...,17,4.6,63.0,https://podcasts.apple.com/us/podcast/badlands...,1627191206,"[Hi listeners! It's Renee, and today we're doi...","[[ck do ufk, 5], [Kat Sale, 5], [Anon8853, 5],..."


In [39]:
### some cleanup and processing

podcasts_df2 = podcasts_df.copy()
podcasts_df2 = podcasts_df2.rename(columns = {'epidsode_descriptions':'episode_descriptions', 'rating':'avg_rating'})
podcasts_df2 = podcasts_df2.explode('ratings_lists')
podcasts_df2

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,ratings_lists
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[RobinFerris, 5]"
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[Pops.99, 1]"
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[ReddEye81, 5]"
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[Keyta7777, 2]"
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[Okkupent, 4]"
...,...,...,...,...,...,...,...,...,...,...,...
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[Monijansand, 5]"
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[trinityangel13, 5]"
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[Kweenkeys, 5]"
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[JoyfulJoyfulWOG, 5]"


In [41]:
podcasts_df2.ratings_lists

0           [RobinFerris, 5]
0               [Pops.99, 1]
0             [ReddEye81, 5]
0             [Keyta7777, 2]
0              [Okkupent, 4]
                ...         
4036        [Monijansand, 5]
4036     [trinityangel13, 5]
4036          [Kweenkeys, 5]
4036    [JoyfulJoyfulWOG, 5]
4036        [Dawn Melody, 5]
Name: ratings_lists, Length: 37269, dtype: object

In [43]:
podcasts_df3 = podcasts_df2.copy()
podcasts_df3[['ratings', 'users']] = podcasts_df3['ratings_lists'].apply(pd.Series)

podcasts_df3 

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,ratings_lists,ratings,users
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[RobinFerris, 5]",RobinFerris,5.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[Pops.99, 1]",Pops.99,1.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[ReddEye81, 5]",ReddEye81,5.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[Keyta7777, 2]",Keyta7777,2.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,"[Okkupent, 4]",Okkupent,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[Monijansand, 5]",Monijansand,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[trinityangel13, 5]",trinityangel13,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[Kweenkeys, 5]",Kweenkeys,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,"[JoyfulJoyfulWOG, 5]",JoyfulJoyfulWOG,5.0


In [46]:
podcasts_df4 = podcasts_df3.copy()
podcasts_df4.episode_descriptions.apply(clean_description)
podcasts_df4

KeyboardInterrupt: ignored

In [ ]:
podcasts_df.to_pickle('re-scraped_podcasts_with_episode_description.pkl')

In [ ]:
#fixing typo in episode_description column
loaded = pd.read_pickle('re-scraped_podcasts_with_episode_description.pkl')
loaded = loaded.rename(columns={'epidsode_description':'episode_description'})
loaded.to_pickle('re-scraped_podcasts_with_episode_description.pkl')

In [ ]:
loaded.episode_description[0]

["We want to make these recorded podcast more directly about RC research, but you’ll see that even that plan gets derailed! The Carbon Cycle and Carbon “sinks” are revisited, as RC gets into an overview of his detailed article “Redemption of the Beast” that he compiled in 2017, which includes numerous quotes and articles about the absolutely vital role of Carbon Dioxide in the biosphere. He digresses, however, in an attempt to defend his friend Graham Hancock, and ends up in a tussle with Normal Guy Mike. First episode of “Randall Rants” to be extracted from this classic RC diatribe? Kosmographia Ep094 The Randall Carlson Podcast with Brothers of the Serpent – Kyle and Russ, Normal Guy Mike, and GeocosmicREX admin Bradley, from 1/10/23.\n In the name of liberty and freedom, we are moving this podcast to our new partner platform! Please join us here: https://www.howtube.com/channels/RandallCarlson\n LINKS:\n Announcements about events and tours: https://randallcarlson.com/tours-and-even

In [ ]:
loaded2 = loaded.episode_description.apply(clean_description)


In [ ]:
loaded2[0]

['we want to make these recorded podcast more directly about rc research but youll see that even that plan gets derailed the carbon cycle and carbon sinks are revisited as rc gets into an overview of his detailed article redemption of the beast that he compiled in  which includes numerous quotes and articles about the absolutely vital role of carbon dioxide in the biosphere he digresses however in an attempt to defend his friend graham hancock and ends up in a tussle with normal guy mike first episode of randall rants to be extracted from this classic rc diatribe kosmographia ep the randall carlson podcast with brothers of the serpent  kyle and russ normal guy mike and geocosmicrex admin bradley from   in the name of liberty and freedom we are moving this podcast to our new partner platform please join us here httpswwwhowtubecomchannelsrandallcarlson  links  announcements about events and tours httpsrandallcarlsoncomtoursandevents  rcs monthly updates on science news and his activities

In [ ]:
get_recent_podcast_episodes(sample_podcast['link'])

AttributeError: ignored

In [ ]:
reload_mine = pd.read_pickle('re-scraped_podcasts.pkl')
reload_mine

,title,producer,genre,description,num_episodes,rating,num_reviews,link,itunes_id
0,Kosmographia,Randall Carlson,Science,All the Randall you can handle,95,4.9,699.0,https://podcasts.apple.com/us/podcast/kosmogra...,1479346591
1,Coffee + Crumbs Podcast,"Katie Blackburn, Jill Atogwe, Ashlee Gadd",Kids & Family,The Coffee + Crumbs Podcast is your companion ...,141,4.8,1700.0,https://podcasts.apple.com/us/podcast/coffee-c...,1093725475
2,Drilled,Critical Frequency,Science,"A true-crime podcast about climate change, hos...",183,4.6,2100.0,https://podcasts.apple.com/us/podcast/drilled/...,1439735906
3,Best Laid Plans,Sarah Hart-Unger,Education,Welcome to Best Laid Plans -- a podcast all ab...,139,4.9,535.0,https://podcasts.apple.com/us/podcast/best-lai...,1525311647
4,The Sports Junkies,Audacy,Sports,"Cakes, Bish, EB and JP on 106.7 the Fan.",500,4.4,1500.0,https://podcasts.apple.com/us/podcast/the-spor...,367825854
...,...,...,...,...,...,...,...,...,...
4107,Oxventure - A Dungeons & Dragons Podcast,Oxventure,Leisure,The Oxventurers Guild presents a DnD real play...,134,5.0,655.0,https://podcasts.apple.com/us/podcast/oxventur...,1528568221
4108,Future of Infrastructure,Microsoft Podcasts,Technology,"Host Jeremy Goldberg, Worldwide Director of Cr...",15,NaN,NaN,https://podcasts.apple.com/us/podcast/future-o...,1674271676
4109,The TWIML AI Podcast (formerly This Week in Ma...,Sam Charrington,Technology,Machine learning and artificial intelligence a...,640,4.7,370.0,https://podcasts.apple.com/us/podcast/the-twim...,1116303051
4110,The Bestie Breakdown,The Bestie Breakdown,Kids & Family,Hi! We're Shay Shull from Mix and Match Mama a...,51,3.8,1400.0,https://podcasts.apple.com/us/podcast/the-best...,1509081920


In [ ]:
reload_his = pd.read_pickle('podcast_dataframe_from_siddgood.pkl')
reload_his

,title,producer,genre,description,num_episodes,rating,num_reviews,link,episode_titles,episode_descriptions
0,History Hyenas with Chris Distefano and Yannis...,RiotCast Network,Comedy,Comedians Yannis Pappas and Chris Distefano go...,98.0,4.9,1300.0,https://podcasts.apple.com/us/podcast/history-...,robert moses was wild kinsey was wild prohibit...,the boys discuss robert moses wild the hyenas ...
1,Curiosity Daily,Westwood One,Education,The award-winning Curiosity Daily podcast from...,300.0,4.8,428.0,https://podcasts.apple.com/us/podcast/curiosit...,maps as weapons and art w justin fornal and em...,explorers justin fornal and emiliano ruprah fr...
2,Spirits,Multitude,History,"A boozy weekly podcast about mythology, legend...",161.0,4.8,1200.0,https://podcasts.apple.com/us/podcast/spirits/...,jacheongbi multimedia mythmaking with carmen m...,this story seems like we lifted it straight ou...
3,The Soundtrack Show,iHeartRadio,TV & Film,The Soundtrack Show is a weekly look at film s...,67.0,4.9,879.0,https://podcasts.apple.com/us/podcast/the-soun...,season intermission the nightmare before chri...,season intermission learn more about your ad...
4,Writing Excuses,"Brandon Sanderson, Mary Robinette Kowal, Dan W...",Arts,"Brandon Sanderson, Mary Robinette Kowal, Howar...",58.0,4.7,775.0,https://podcasts.apple.com/us/podcast/writing-...,economics realism vs ruleofcool sequencing you...,your hosts brandon dan howard and mahtab econ...
...,...,...,...,...,...,...,...,...,...,...
4298,Dark Histories,Ben Cutmore,History,Bi-weekly narratives on the unsolved and the u...,72.0,4.9,287.0,https://podcasts.apple.com/us/podcast/dark-his...,the yatton demoniac george lukins the sarah du...,sir when you can spare room in your gazette i ...
4299,Red Scare,Red Scare,Society & Culture,Red Scare is a cultural commentary podcast hos...,99.0,4.0,1200.0,https://podcasts.apple.com/us/podcast/red-scar...,bore carnival teaser pest control greta fg thu...,the ladies discuss the online ya wars the cuts...
4300,The Amelia Project,Imploding Fictions,Fiction,The Amelia Project is a secret agency that fak...,39.0,4.9,386.0,https://podcasts.apple.com/us/podcast/the-amel...,richard season finale ant cleo cole niseag b...,i always obey the dice episode season fina...
4301,L.A. Not So Confidential,Dr. Scott & Dr. Shiloh,True Crime,An L.A. based podcast brought to you by two fo...,36.0,4.8,151.0,https://podcasts.apple.com/us/podcast/l-a-not-...,killer nurses the fear of clowns intimate part...,dr scott and dr shiloh delve into the world of...


In [ ]:
reload_his.iloc[1].episode_descriptions

'explorers justin fornal and emiliano ruprah from the science channels unexplained and unexplored discuss the surprising roles that maps have played throughout history youll also learn about why people fidget learn about whether humans have an internal compass how long a flu virus can last in the environment and whether creativity and mental illness actually go hand in hand forrest galante host of extinct or alive on animal planet explains how to become an adventurer survive extreme conditions and find extinct animals then youll learn about how cryptomnesia can make you plagiarize without realizing it learn about why the amount of light you see affects how cold you feel whats inside a kangaroos pouch and foods that can curb the effects of sleep deprivation popular mathematics writer and cornell university professor steven strogatz discusses the difference between calculus and algebra and what calculus can do that other math cant plus learn about how distraction can alter your memory an